# Projeto Final

Para esse projeto foi considerado a base de dados em Português do Brasil de reviews disponíveis no IMDB.

Esse site contém uma ampla variedade de catálogos de filmes em que as pessoas cadastradas podem realizar comentários com suas opiniões.

Essa base de dados está disponível em: 
* https://huggingface.co/datasets/celsowm/imdb-reviews-pt-br

In [1]:
print("Hello World!")

Hello World!


In [ ]:
!ls -la

total 40068
drwxr-xr-x 2 andrevrc andrevrc     4096 Jun 26 17:06 .
drwxr-xr-x 3 andrevrc andrevrc     4096 Jun 26 17:03 ..
-rw-r--r-- 1 andrevrc andrevrc      701 Jun 26 17:15 projeto_final_review_analysis.ipynb
-rw-r--r-- 1 andrevrc andrevrc 41015476 Nov 15  2023 train-00000-of-00001-34e817339f28df45.parquet
